# Levenshtein distance and spelling corrections

1. Use the corpus from exercise no. 1.

In [1]:
from datasets import load_dataset

dataset = load_dataset("clarin-knext/fiqa-pl", "corpus")

2. Use SpaCy tokenizer API to tokenize the text in the documents.

In [2]:
import spacy
from spacy.lang.pl import Polish

nlp = Polish()

tokenized_texts = []
for doc in dataset['corpus']['text']:
    spacy_doc = nlp(doc)
    tokens = [token.text.lower() for token in spacy_doc if token.is_alpha]
    tokenized_texts.append(tokens)

print("First document tokens:", tokenized_texts[0])

First document tokens: ['nie', 'mówię', 'że', 'nie', 'podoba', 'mi', 'się', 'też', 'pomysł', 'szkolenia', 'w', 'miejscu', 'pracy', 'ale', 'nie', 'możesz', 'oczekiwać', 'że', 'firma', 'to', 'zrobi', 'szkolenie', 'pracowników', 'to', 'nie', 'ich', 'praca', 'oni', 'tworzą', 'oprogramowanie', 'być', 'może', 'systemy', 'edukacyjne', 'w', 'stanach', 'zjednoczonych', 'lub', 'ich', 'studenci', 'powinny', 'trochę', 'martwić', 'się', 'o', 'zdobycie', 'umiejętności', 'rynkowych', 'w', 'zamian', 'za', 'ich', 'ogromne', 'inwestycje', 'w', 'edukację', 'zamiast', 'wychodzić', 'z', 'tysiącami', 'zadłużonych', 'studentów', 'i', 'narzekać', 'że', 'nie', 'są', 'do', 'niczego', 'wykwalifikowani']


3. Compute frequency list for each of the processed files.

In [3]:
from collections import Counter

frequency_lists = []

for tokens in tokenized_texts:
    frequency_list = Counter(tokens)
    frequency_lists.append(frequency_list)

print("Frequency list for the first document:", frequency_lists[0])

Frequency list for the first document: Counter({'nie': 5, 'w': 4, 'że': 3, 'ich': 3, 'się': 2, 'to': 2, 'mówię': 1, 'podoba': 1, 'mi': 1, 'też': 1, 'pomysł': 1, 'szkolenia': 1, 'miejscu': 1, 'pracy': 1, 'ale': 1, 'możesz': 1, 'oczekiwać': 1, 'firma': 1, 'zrobi': 1, 'szkolenie': 1, 'pracowników': 1, 'praca': 1, 'oni': 1, 'tworzą': 1, 'oprogramowanie': 1, 'być': 1, 'może': 1, 'systemy': 1, 'edukacyjne': 1, 'stanach': 1, 'zjednoczonych': 1, 'lub': 1, 'studenci': 1, 'powinny': 1, 'trochę': 1, 'martwić': 1, 'o': 1, 'zdobycie': 1, 'umiejętności': 1, 'rynkowych': 1, 'zamian': 1, 'za': 1, 'ogromne': 1, 'inwestycje': 1, 'edukację': 1, 'zamiast': 1, 'wychodzić': 1, 'z': 1, 'tysiącami': 1, 'zadłużonych': 1, 'studentów': 1, 'i': 1, 'narzekać': 1, 'są': 1, 'do': 1, 'niczego': 1, 'wykwalifikowani': 1})


4. Aggregate the result to obtain one global frequency list. This frequency list gives you unigram statistics of the words appearing in the corpus.

In [4]:
global_frequency_list = Counter()

for tokens in tokenized_texts:
    global_frequency_list.update(tokens)

print("Global frequency list:", global_frequency_list)
print("Top 10 most common words:", global_frequency_list.most_common(10))

Global frequency list: Counter({'w': 175366, 'nie': 131482, 'i': 126839, 'na': 119047, 'to': 116468, 'z': 96953, 'jest': 93246, 'że': 90021, 'się': 85843, 'do': 66601, 'jeśli': 52003, 'ale': 41308, 'a': 41264, 'o': 38203, 'są': 35435, 'jak': 33672, 'lub': 33216, 'za': 32317, 'aby': 31053, 'od': 29829, 'co': 29702, 'może': 26370, 'po': 25715, 'tak': 25634, 'dla': 25462, 'które': 23983, 'możesz': 23280, 'tego': 21938, 'tym': 21608, 'czy': 20285, 'ma': 20284, 'być': 19267, 'ponieważ': 18957, 'przez': 17833, 'usd': 17524, 'tylko': 17262, 'więc': 17177, 'niż': 16827, 'ich': 14570, 'pieniądze': 13969, 'więcej': 13838, 'gdy': 13534, 'jako': 12980, 'pieniędzy': 12278, 'który': 12139, 'masz': 11710, 'będzie': 11694, 'mogą': 11679, 'firmy': 10725, 'również': 10531, 'akcji': 10417, 'bardzo': 10180, 'ci': 9984, 'prostu': 9782, 'nawet': 9727, 'lat': 9509, 'mają': 9298, 'wiele': 9223, 'ten': 8990, 'te': 8780, 'ze': 8616, 'kiedy': 8595, 'bez': 8281, 'akcje': 8117, 'teraz': 8101, 'roku': 8043, 'prawdo

5. Apply a distortion function to the queries part of the corpus. In each query draw randomly one word and change one letter in the word to some other letter.

In [5]:
query = load_dataset("clarin-knext/fiqa-pl", "queries")
queries = query['queries'].to_pandas()

In [6]:
import random
import string

def distort_query(query):
    words = query.split()
    if words:
        word_idx = random.randint(0, len(words) - 1)
        word = words[word_idx]
        if len(word) > 1:
            char_idx = random.randint(0, len(word) - 1)
            new_char = random.choice([chr(i) for i in range(97, 123) if chr(i) != word[char_idx]])
            distorted_word = word[:char_idx] + new_char + word[char_idx + 1:]
            words[word_idx] = distorted_word
    return ' '.join(words)

df = queries

df['distorted_query'] = df['text'].apply(distort_query)
print(df)

       _id title                                               text  \
0        0        Co jest uważane za wydatek służbowy w podróży ...   
1        4        Wydatki służbowe - ubezpieczenie samochodu pod...   
2        5                        Rozpoczęcie nowego biznesu online   
3        6           „Dzień roboczy” i „termin płatności” rachunków   
4        7        Nowy właściciel firmy – Jak działają podatki d...   
...    ...   ...                                                ...   
6643  4102        Jak mogę ustalić, czy moja stopa zwrotu jest „...   
6644  3566        Gdzie mogę kupić akcje, jeśli chcę zainwestowa...   
6645    94        Wykorzystywanie punktów kart kredytowych do op...   
6646  2551        Jak znaleźć tańszą alternatywę dla tradycyjnej...   
6647  2399        Skąd strony internetowe uzyskują informacje o ...   

                                        distorted_query  
0     Co jast uważane za wydatek służbowy w podróży ...  
1     Wydatki służbowe - ubezpi

6. Compute nDCG@10 for the distorted queris, using the same approach as in lab 2. This result will be the baseline for the other methods.

In [7]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")
print(es.info())

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x00000239B3DD25D0>: Failed to establish a new connection: [WinError 10061] Nie można nawiązać połączenia, ponieważ komputer docelowy aktywnie go odmawia))

In [2]:
from datasets import load_dataset
import numpy as np
import pandas as pd

qa_dataset = load_dataset("clarin-knext/fiqa-pl-qrels")
test_data = qa_dataset['test']
qrels = test_data.to_pandas()


def calc_ndcg_k(true, predicted):
    dcg = np.sum(predicted / np.log2(np.arange(2, len(true) + 2)))
    idcg = np.sum(true / np.log2(np.arange(2, len(true) + 2)))
    return dcg / idcg if idcg > 0 else 0


def count_ndcg(index, maps, query_option):
    max_matches = qrels.groupby('query-id')['corpus-id'].count().rename('count')

    ndcg_total = 0
    num_queries = 0

    for query_id in qrels['query-id'].unique():
        query = queries.loc[queries['_id'] == str(query_id), query_option].iloc[0]

        query_blank = {
            "match": {
                maps: {
                    "query": query
                }
            }
        }

        resp = es.search(index=index, query=query_blank)
        corpus_ids = qrels.loc[qrels['query-id'] == query_id, 'corpus-id'].tolist()
        matches = min(max_matches.loc[query_id], 10)
        true = np.array([1 if i < matches else 0 for i in range(10)])

        predicted = np.zeros(10)

        for i, hit in enumerate(resp['hits']['hits'][:10]):
            predicted[i] = 1 if int(hit['_id']) in corpus_ids else 0
        ndcg_total += calc_ndcg_k(true, predicted)
        num_queries += 1

    return ndcg_total / num_queries if num_queries > 0 else 0

In [32]:
mean_ndcg = count_ndcg("fiqa-pl-index", "with_synonyms", "distorted_query")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")

The average NDCG is 0.18332102219754928 for the index with synonyms and lemmatization.


In [33]:
mean_ndcg = count_ndcg("fiqa-pl-index", "without_synonyms", "distorted_query")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")

The average NDCG is 0.18335904228591737 for the index with synonyms and lemmatization.


In [34]:
mean_ndcg = count_ndcg("fiqa-pl-index_no_lem", "with_synonyms_no_lem", "distorted_query")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")

The average NDCG is 0.13531053613641197 for the index with synonyms and lemmatization.


In [35]:
mean_ndcg = count_ndcg("fiqa-pl-index_no_lem", "without_synonyms_no_lem", "distorted_query")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")

The average NDCG is 0.13503701871853538 for the index with synonyms and lemmatization.


7. Install Morfeusz (Binding dla Pythona) and use it to find all words from the queries that do not appear in that dictionary. Only these words should be corected in the next step.

In [6]:
import morfeusz2
import pandas as pd

morfeusz = morfeusz2.Morfeusz()

def is_recognized(word):
    """# Function to check if a word is recognized by Morfeusz"""
    analysis = morfeusz.analyse(word)
    return any("ign" not in interpretation[2] for interpretation in analysis)

queries['unrecognized_words'] = queries['distorted_query'].apply(
    lambda query: [word for word in query.split() if not is_recognized(word)]
)

print(queries[['distorted_query', 'unrecognized_words']])

                                        distorted_query unrecognized_words
0     Co jesd uważane za wydatek służbowy w podróży ...             [jesd]
1     Wydatki służbowe - ubezpieczenie samochodu pod...         [wydarzyx]
2                     Rozpoczęcie ntwego biznesu online           [ntwego]
3        „Dzień roboczy” i „termin płatności” rarhunków        [rarhunków]
4     Nowy właściciel firmy – Jak działają podatki d...                 []
...                                                 ...                ...
6643  Jak mogę ustaldć, czy moja stopa zwrotu jest „...                 []
6644  Gdzie mogn kupić akcje, jeśli chcę zainwestowa...             [mogn]
6645  Wykorzystywanie punktów kart kredytowych do op...                 []
6646  Jak znalewć tańszą alternatywę dla tradycyjnej...          [znalewć]
6647  Skąd strony internetowe uzyskują informacje o ...                 []

[6648 rows x 2 columns]


8. Use Levenshtein distance and the frequency list, to determine the most probable correction of the words in the queries that were identified as invalid. (Note: You don't have to apply the distance directly. Any method that is more efficient than scanning the dictionary will be appreciated.)

In [ ]:
global_frequency_list = global_frequency_list

def generate_candidates(word):
    """Function to generate possible spelling corrections for a given word by creating
    variations through deletion, insertion, and substitution of letters."""
    letters = 'abcdefghijklmnopqrstuvwxyząćęłńóśźż'  # Alphabet used for corrections.
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]  # Create all split points for the word.
    deletes = [L + R[1:] for L, R in splits if R]  # Create candidates by deleting one character.
    inserts = [L + c + R for L, R in splits for c in letters]  # Insert each letter at each split.
    substitutes = [L + c + R[1:] for L, R in splits if R for c in letters]  # Substitute each letter.
    candidates = set(deletes + inserts + substitutes)  # Combine all variations into a set of candidates.
    return candidates

def correct_word(word, frequency_list):
    """Function to correct a word by choosing the most frequent valid candidate from the frequency list."""
    candidates = generate_candidates(word)
    valid_candidates = [candidate for candidate in candidates if candidate in frequency_list]
    if valid_candidates:
        return max(valid_candidates, key=lambda candidate: frequency_list[candidate])
    else:
        return word

def correct_invalid_words(query):
    """Function to correct unrecognized words in a query using the frequency list."""
    corrected_words = []
    for word in query.split():
        if not is_recognized(word):
            corrected_words.append(correct_word(word, frequency_list))
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

queries['corrected_query'] = queries['distorted_query'].apply(correct_invalid_words)
print(queries[['distorted_query', 'corrected_query']])

                                        distorted_query  \
0     Co jesd uważane za wydatek służbowy w podróży ...   
1     Wydatki służbowe - ubezpieczenie samochodu pod...   
2                     Rozpoczęcie ntwego biznesu online   
3        „Dzień roboczy” i „termin płatności” rarhunków   
4     Nowy właściciel firmy – Jak działają podatki d...   
...                                                 ...   
6643  Jak mogę ustaldć, czy moja stopa zwrotu jest „...   
6644  Gdzie mogn kupić akcje, jeśli chcę zainwestowa...   
6645  Wykorzystywanie punktów kart kredytowych do op...   
6646  Jak znalewć tańszą alternatywę dla tradycyjnej...   
6647  Skąd strony internetowe uzyskują informacje o ...   

                                        corrected_query  
0     Co jest uważane za wydatek służbowy w podróży ...  
1     Wydatki służbowe - ubezpieczenie samochodu pod...  
2                     Rozpoczęcie ntwego biznesu online  
3        „Dzień roboczy” i „termin płatności” rarhunków  
4

9. Compute nDCG@10 for your implementation of the spelling correction method.

In [25]:
import time
start = time.time()
mean_ndcg = count_ndcg("fiqa-pl-index", "with_synonyms", "corrected_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.19277747434339354 for the index with synonyms and lemmatization.
Time:  50.873374700546265


In [26]:
import time
start = time.time()
mean_ndcg = count_ndcg("fiqa-pl-index", "without_synonyms", "corrected_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.19274989956477068 for the index with synonyms and lemmatization.
Time:  48.362220287323


In [27]:
import time
start = time.time()
mean_ndcg = count_ndcg("fiqa-pl-index_no_lem", "with_synonyms_no_lem", "corrected_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.13933154759766475 for the index with synonyms and lemmatization.
Time:  47.0933792591095


In [28]:
import time
start = time.time()
mean_ndcg = count_ndcg("fiqa-pl-index_no_lem", "with_synonyms_no_lem", "corrected_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.13933154759766475 for the index with synonyms and lemmatization.
Time:  43.47527313232422


10. Use ElasticSearch's fuzzy match and compute nDCG@10 for this approach.

In [ ]:
def count_ndcg_fuzzy(index, maps, query_option):
    max_matches = qrels.groupby('query-id')['corpus-id'].count().rename('count')

    ndcg_total = 0
    num_queries = 0

    for query_id in qrels['query-id'].unique():
        query_text = queries.loc[queries['_id'] == str(query_id), query_option].iloc[0]

        fuzzy_query = {
            "match": {
                maps: {
                    "query": query_text,
                    "fuzziness": "AUTO" 
                }
            }
        }

        resp = es.search(index=index, query=fuzzy_query, size=10)
        corpus_ids = qrels.loc[qrels['query-id'] == query_id, 'corpus-id'].tolist()
        matches = min(max_matches.loc[query_id], 10)
        true = np.array([1 if i < matches else 0 for i in range(10)])
        
        predicted = np.zeros(10)
        for i, hit in enumerate(resp['hits']['hits'][:10]):
            predicted[i] = 1 if int(hit['_id']) in corpus_ids else 0
        
        ndcg_total += calc_ndcg_k(true, predicted)
        num_queries += 1

    return ndcg_total / num_queries if num_queries > 0 else 0

In [31]:
import time
start = time.time()
mean_ndcg = count_ndcg_fuzzy("fiqa-pl-index", "with_synonyms", "distorted_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")
print("Time: ", stop - start)


The average NDCG is 0.16977313379639597 for the index with synonyms and lemmatization.
Time:  46.96927833557129


In [ ]:
import time
start = time.time()
mean_ndcg = count_ndcg_fuzzy("fiqa-pl-index", "without_synonyms", "distorted_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index without synonyms and lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.16756315263474295 for the index without synonyms and lemmatization.


In [ ]:
import time
start = time.time()
mean_ndcg = count_ndcg_fuzzy("fiqa-pl-index_no_lem", "with_synonyms_no_lem", "distorted_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and no lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.14288072402171625 for the index with synonyms and no lemmatization.


In [ ]:
import time
start = time.time()
mean_ndcg = count_ndcg_fuzzy("fiqa-pl-index_no_lem", "without_synonyms_no_lem", "distorted_query")
stop = time.time()
print(f"The average NDCG is {mean_ndcg} for the index without synonyms and no lemmatization.")
print("Time: ", stop - start)

The average NDCG is 0.14280628911914078 for the index without synonyms and no lemmatization.


11. Compare the results of baseline with the 2 implemented methods. Take into account the nDCG score and the performance of the methods.

Jeżeli chodzi o czas wykonania to nieznacznie różnią się od siebie w obydwu przypadkach, elasticsearch jest krótszy. 
Natomiast wyniki zwrócone w przypadku własnej implementacji są lepsze niż przy fuzzy.
Dla fuzzy z lematyzacją wyniki ulegają pogorszeniu a w drugim przypadku (z lematyzacją) polepszone są.

12. Use an LLM of your choice (you can use Bielik) to fix 30 first queries from the distorted set and compare the results manually with the method based on the Levenshtein distance.

In [ ]:
queries_distorted = queries[['distorted_query']].head(30)
for query in queries_distorted['distorted_query']:
    print(query)

Co jesd uważane za wydatek służbowy w podróży służbowej?
Wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzyx się podczas podróży służbowej
Rozpoczęcie ntwego biznesu online
„Dzień roboczy” i „termin płatności” rarhunków
Nowy właściciel firmy – Jak działają podatki dla fiemy i osoby fizycznej?
Hobby krntra biznes
Czeki osobiste zampast firmowych
Czy amerykfński kodeks podatkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?
Jak mogę zarejestrować firmę w Wielkiej Brytanii bez podawania adresu fiwmy?
Czlm są „podstawy biznesowe”?
Strata nd inwestycjach biznesowych z poprzedniego roku
Jak mogę oszccować podatki biznesowe / opłaty za zgłoszenie dla firmy, która ma 0 USD dochodu?
Czy zakup samochodu dla firmy za pomocą kredytu biznesowego zostałby uknany za wydatek biznesowy?
Odliczanie ptrat biznesowych z ostatnich lat (nieudokumentowanych)
30% udziału w bizxesie
mdbieranie wydatków służbowych z osobistej karty kr

Wyniki Bielik:
- jesd -> jest
- wydarzyx -> wydarzył się
- ntwego -> nowego
- rarhunków -> rachunków
- fiemy -> firmy
- krntra -> kontra
- zampast -> zamiast
- amerykfński -> amerykański
- fiwmy -> firmy
- Czlm -> Czym
- nd -> na
- oszccować -> oszacować
- uknany -> uznany
- ptrat -> wydatków
- bizxesie -> biznesie
- mdbieranie -> odbieranie
- płatnoścd -> płatności
- cpasu -> czasu
- karry -> karty
- pouocznej -> po urodzeniu
- legalzą -> legalną
- milionereh -> milionerem
- małez -> małej
-dulego -> dużego
- douyczące -> dotyczące
- kirmy -> firmy
- pieniędhy -> pieniędzy
- Jah -> Jak
- Korepetycje -> Korepetycje
- uodatki -> podatki

When specifying exactly how to modify the query, such as by correcting a single letter, the model achieved a success rate of 28 out of 30. However, when given a more general instruction to "correct the query," the success rate dropped to 17 out of 30. This suggests that providing more precise instructions significantly improves the accuracy of the corrections.

## Conclusion

Dystrybucja słów w korpusie:
Dystrybucja może być nierównomierna, co oznacza, że ​​niektóre słowa występują znacznie częściej, co może mieć wpływ na wydajność wyszukiwania, ponieważ bardziej popularne terminy mogą być zbyt ogólne.

Wydajność metody w porównaniu do ElasticSearch:
Porównując własną metodę z ElasticSearch, można ocenić różnice w szybkości i dokładności wyników. ElasticSearch jest zoptymalizowany pod kątem szybkiego wyszukiwania w dużych korpusach, więc jest zwykle bardziej wydajny. ElasticSearch działa szybciej niż własna implementacja.

Porównanie wyników uzyskanych własną metodą i z ElasticSearch:
Metoda własna daje lepsze wyniki, gdy mamy do czynienia z lematyzacją, bez lematyzacji Elasticsearch działa lepiej.

Możliwość LLM korygowania niepoprawnych zapytań:
Model LLM może skutecznie korygować niepoprawne zapytania, zwłaszcza jeśli są to typowe błędy językowe lub literówki. W przypadku bardziej szczegółowych błędów (np. w terminologii branżowej) wydajność LLM może być niższa.